In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
import keras
import keras.ops as K
from keras.layers import Input, Flatten, Dense
from keras.optimizers import Adam
from keras.metrics import BinaryAccuracy

# from keras.models import Sequential
from deel.lip.model import Sequential

from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.activations import GroupSort, GroupSort2
from deel.lip.losses import HKR, KR, HingeMargin, MulticlassHKR, MulticlassKR

import numpy as np
import decomon

from data_processing import load_data, select_data_for_radius_evaluation_MNIST08
from radius_evaluation_tools import compute_binary_certificate, starting_point_dichotomy

In [3]:
x_train, x_test, y_train, y_test, y_test_ord = load_data("MNIST08")

In [13]:
model_path = "/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output.keras"
model = keras.models.load_model(model_path)
model.compile(
   
    loss=HKR(
        alpha=10.0, min_margin=1.0
    ),  # HKR stands for the hinge regularized KR loss
    metrics=[
        # KR,  # shows the KR term of the loss
        HingeMargin(min_margin=1.0),  # shows the hinge term of the loss
    ],
    optimizer=Adam(learning_rate=0.001),)

In [12]:
model_bis_path = "/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output_converted_4logits.keras"
model_bis = keras.models.load_model(model_bis_path)
model_bis.compile(
        # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
        # note also in the case of lipschitz networks, more robustness require more parameters.
        loss=MulticlassHKR(alpha=100, min_margin=0.25),
        optimizer=Adam(1e-4),
        metrics=["accuracy", MulticlassKR()],)

In [14]:
from decomon.layers import DecomonLayer
from decomon.models import clone
from lipschitz_custom_tools import affine_bound_groupsort_output_keras, affine_bound_sqrt_output_keras, affine_bound_square_output_keras
from decomon.perturbation_domain import BallDomain
from decomon import get_lower_noise, get_range_noise, get_upper_noise
import torchattacks

In [15]:
import pdb

In [16]:
images, labels, idx_list = select_data_for_radius_evaluation_MNIST08(x_test, y_test_ord, model_bis)

torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])


In [17]:
class DecomonGroupSort2(DecomonLayer):
    layer : GroupSort2
    increasing = True
    def get_affine_bounds(self, lower, upper):
        (W_low, b_low), (W_up, b_up) = affine_bound_groupsort_output_keras(lower, upper)
        return W_low, b_low, W_up, b_up

In [53]:
lip_certificate = compute_binary_certificate(images, model)

In [19]:
def compute_Decomon_certificates(i, epsilon, inputs, model):
    perturbation_domain = BallDomain(eps=epsilon, p=2)
    decomon_model = clone(model, mapping_keras2decomon_classes={GroupSort2:DecomonGroupSort2}, final_ibp=True, final_affine=False, perturbation_domain=perturbation_domain)
    upper_test_ = get_upper_noise(decomon_model,  inputs[i:i+1], eps=epsilon, p=2)[:, 0]
    lower_test_ = get_lower_noise(decomon_model, inputs[i:i+1], eps=epsilon, p=2)[:, 0]
    return lower_test_, upper_test_

In [56]:
print(compute_Decomon_certificates(5, 1.0502, images.cpu().detach().numpy(), model))

(array([-5.574271], dtype=float32), array([-0.04252386], dtype=float32))


In [61]:
print(compute_Decomon_certificates(5, 1.0502, images, model))

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [62]:
def single_compute_decomon_radius(idx, images, targets, model, n_iter = 10):
    image = images[idx:idx+1]
    target = targets[idx:idx+1]
    # certificate = certificates[idx:idx+1]
    # We use dichotomy algorithm to fine the smallest optimistic radius
    # We start from the closest point with different class
    d_up = starting_point_dichotomy(idx, images, targets)
    eps_working = d_low = 0
    for _ in range(n_iter):
        eps_current = (d_up+d_low)/2
        print(eps_current)
        perturbation_domain = BallDomain(eps=eps_current, p=2)
        decomon_model = clone(model, mapping_keras2decomon_classes={GroupSort2:DecomonGroupSort2}, final_ibp=True, final_affine=False, perturbation_domain=perturbation_domain)
        upper = get_upper_noise(decomon_model,  image.cpu().detach().numpy(), eps=eps_current, p=2)[:, 0]
        lower = get_lower_noise(decomon_model, image.cpu().detach().numpy(), eps=eps_current, p=2)[:, 0]

        if (target==0 and upper<=0) or (target==1 and lower<=0):
            print("working", target, upper, lower)
            eps_working = d_low = eps_current
        else:
            print("not working", target, upper, lower)
            d_up = eps_current
    return eps_working

In [63]:
idx = 5
eps = single_compute_decomon_radius(idx, images, labels, model)
print(lip_certificate[idx], eps)

tensor(4.2339, device='cuda:0')
not working tensor([0], device='cuda:0') [24.33216] [-21.806156]
tensor(2.1169, device='cuda:0')
not working tensor([0], device='cuda:0') [7.315631] [-9.606468]
tensor(1.0585, device='cuda:0')
not working tensor([0], device='cuda:0') [0.00775731] [-5.5941496]
tensor(0.5292, device='cuda:0')
working tensor([0], device='cuda:0') [-2.4918537] [-4.4621453]
tensor(0.7939, device='cuda:0')
working tensor([0], device='cuda:0') [-1.465566] [-5.006468]
tensor(0.9262, device='cuda:0')
working tensor([0], device='cuda:0') [-0.7788049] [-5.290392]
tensor(0.9923, device='cuda:0')
working tensor([0], device='cuda:0') [-0.38893855] [-5.439641]
tensor(1.0254, device='cuda:0')
working tensor([0], device='cuda:0') [-0.1932382] [-5.514635]
tensor(1.0419, device='cuda:0')
working tensor([0], device='cuda:0') [-0.09280509] [-5.554384]
tensor(1.0502, device='cuda:0')
working tensor([0], device='cuda:0') [-0.04254037] [-5.5742655]
tensor(3.6832, device='cuda:0', grad_fn=<Selec

In [46]:
eps

tensor(1.0502, device='cuda:0')

In [ ]:
box = np.zeros((1,2,1,28,28))

In [ ]:
box[:,1] += 0.1

In [ ]:
decomon_model.predict(box)

In [ ]:
class Sqrt(keras.layers.Layer):
    def call(self, inputs):
        return K.sqrt(inputs)

class Square(keras.layers.Layer):
    def call(self, inputs):
        return K.square(inputs)

In [ ]:
class DecomonSqrt(DecomonLayer):
    layer : Sqrt
    increasing = True
    def get_affine_bounds(self, lower, upper):
        (W_low, b_low), (W_up, b_up) = affine_bound_sqrt_output_keras(lower, upper)
        return W_low, b_low, W_up, b_up
class DecomonSquare(DecomonLayer):
    layer : Square
    def get_affine_bounds(self, lower, upper):
        (W_low, b_low), (W_up, b_up) = affine_bound_square_output_keras(lower, upper)
        return W_low, b_low, W_up, b_up
    def forward_ibp_propagate(self, lower, upper):
        return self.layer(lower), self.layer(upper)

In [ ]:
sqrt = Sqrt()

In [ ]:
sqrt.predict(4)